<a href="https://colab.research.google.com/github/chsin1/869_course/blob/main/Session_5_ML_for_Volatility_Surface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the example in John Hull's textbook "Machine Learning in Business"

In practice, traders use the Black-Scholes-Merton formula
$$c=Se^{-qT} N(d_1)-Ke^{-rT}N(d_2) $$
$$ d_1= \frac{\ln(S/K)+(r-q-\sigma^2/2)T}{\sigma \sqrt{T}}, d_2=d_1-\sigma\sqrt{T} $$
as a price quotation tool: for a market price $c$, there is a unique constant $\sigma$ such that the B-S-M formula holds. This is called the implied Black-Schols volatility.

Market data are usually quoted as implied volatilities. Because of the 1-1 relationship between price and volatility, predicting price is equivalent to predicting implied volatility!

For any given time to maturity $T$ and strike price $K$, we can think the implied volatility as a function of $T$ and $K$:
$$ \sigma=\sigma(T,K)$$


As times changes, time to maturity $T$ will change, and spot price of the underlying asset also changes. In this example, it is  ASSUMED that the change in implied volatility can be best represented by
$$\Delta \sigma = \frac{\Delta S}{S} \frac{a+b\delta + c\delta^2}{\sqrt{T}}$$
where
$$ \delta \equiv \frac{\partial c}{\partial S} = e^{-qT}N(d_1)$$
and $a, b, c$ are constants, and can be estimated through the following regression
$$\Delta c = \alpha + a \frac{R}{\sqrt{T}} + b\frac{R\delta}{\sqrt{T}} + c \frac{R\delta^2}{\sqrt{T}} + \cdots $$
where $R=\Delta S/S$.

For more on implied volatility surface, see Daglish, Hull, and Suo: "em Volatility surfaces theory rules of thumb and empirical evidence"

For more detailed discussion of this example, see Cao, Chen, and Hull: "A neural network approach to understanding implied volatility movements"

In [ ]:
#This program takes about 60 minutes to run
#Loading Package
import time
import numpy as np
import scipy as sci
import scipy.io as sio
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow import keras


from numpy.random import seed
seed(42)


import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import clear_output


# We first download the volatility data from Professor Hull's webpage

In [ ]:
!wget https://www-2.rotman.utoronto.ca/~hull/MLThirdEditionFiles/VolatilitySurfaceExample/Implied_Volatility_Data_vFinal.csv

--2025-02-03 03:44:50--  https://www-2.rotman.utoronto.ca/~hull/MLThirdEditionFiles/VolatilitySurfaceExample/Implied_Volatility_Data_vFinal.csv
Resolving www-2.rotman.utoronto.ca (www-2.rotman.utoronto.ca)... 128.100.40.235
Connecting to www-2.rotman.utoronto.ca (www-2.rotman.utoronto.ca)|128.100.40.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5809460 (5.5M) [application/octet-stream]
Saving to: ‘Implied_Volatility_Data_vFinal.csv’

Implied_Volatility_ 100%[===================>]   5.54M  10.9MB/s    in 0.5s    

2025-02-03 03:44:50 (10.9 MB/s) - ‘Implied_Volatility_Data_vFinal.csv’ saved [5809460/5809460]



In [ ]:
# load raw data
raw = pd.read_csv('Implied_Volatility_Data_vFinal.csv')
# check the raw data
print("Size of the dataset (row, col): ", raw.shape)
raw.head(5)

Size of the dataset (row, col):  (125700, 5)


,Date,SPX Return,Time to Maturity in Year,Delta,Implied Volatility Change
0,20140630,0.006678,0.184,0.745927,0.008462
1,20140630,0.006678,2.252,0.286911,0.002024
2,20140630,0.006678,0.124,0.902941,0.019707
3,20140630,0.006678,2.144,0.910926,0.006424
4,20140630,0.006678,1.412,0.885760,0.005673


## Construct the 3 variables for Regression Approach

In [ ]:
# construct the 3 variables for regression
raw['x1'] = raw['SPX Return'] / np.sqrt(raw['Time to Maturity in Year'])
raw['x2'] = raw['SPX Return'] / np.sqrt(raw['Time to Maturity in Year']) * raw['Delta']
raw['x3'] = raw['x2'] * raw['Delta']

# Put the X and Y variable in data frame for regression
y = raw['Implied Volatility Change']
X = raw[['x1', 'x2', 'x3','SPX Return','Time to Maturity in Year','Delta']]

In [ ]:
X

,x1,x2,x3,SPX Return,Time to Maturity in Year,Delta
0,0.015568,0.011612,0.008662,0.006678,0.184,0.745927
1,0.004450,0.001277,0.000366,0.006678,2.252,0.286911
2,0.018964,0.017123,0.015461,0.006678,0.124,0.902941
3,0.004561,0.004154,0.003784,0.006678,2.144,0.910926
4,0.005620,0.004978,0.004409,0.006678,1.412,0.885760
...,...,...,...,...,...,...
125695,0.008007,0.002122,0.000562,0.003823,0.228,0.265052
125696,0.010075,0.009507,0.008970,0.003823,0.144,0.943591
125697,0.015608,0.002315,0.000343,0.003823,0.060,0.148294
125698,0.006557,0.006016,0.005519,0.003823,0.340,0.917499


In [ ]:
# Divide data into training set and test set(note that random seed is set)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=100)

# Divide training set into training and validation set
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.25,random_state=100)

## Feature Scaling

In [ ]:
# Scale features based on Z-Score
scaler = StandardScaler()
scaler.fit(X_train)


X_scaled_train = scaler.transform(X_train)
X_scaled_vals = scaler.transform(X_val)
X_scaled_test = scaler.transform(X_test)
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)
y_test = np.asarray(y_test)

In [ ]:
X_scaled_train.shape

(75420, 6)

In [ ]:
X_scaled_train[0:5, :]

array([[-0.76891115, -0.10677003, -0.00863333, -0.94407032, -0.38490458,
        -1.93041082],
       [-0.25423855, -0.07070529, -0.01215069, -0.20412281, -0.64097773,
        -1.57580193],
       [ 0.47778621,  0.60160057,  0.60342402,  0.30508336, -0.66344029,
         0.58513584],
       [-2.56724945, -0.94943864, -0.28743962, -1.75903042, -0.66344029,
        -1.35955708],
       [ 0.92109161,  1.12064003,  1.10743618,  0.54230472, -0.68590284,
         0.56879434]])

In [ ]:
X_scaled_train[:, 0:3]

array([[-0.76891115, -0.10677003, -0.00863333],
       [-0.25423855, -0.07070529, -0.01215069],
       [ 0.47778621,  0.60160057,  0.60342402],
       ...,
       [ 0.87240506,  0.94876312,  0.83939087],
       [-2.0433805 , -1.45123875, -0.85717121],
       [-0.10863139, -0.10810545, -0.11113695]])

## Run Regression

In [ ]:
# Run the regression on the training data
lr = LinearRegression(fit_intercept=False)
lr.fit(X_scaled_train[:,:3], y_train)

# Get the prediction
y_pred = lr.predict(X_scaled_test[:,:3])

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

print('Test loss (MSE):', mse)

Test loss (MSE): 7.423167462421238e-05


## ML Approach

In [ ]:
# Create ML Model
# Sequential function allows you to define your Neural Network in sequential order
# Within Sequential, use Dense function to define number of nodes, activation function and other related parameters
# For more information regrading to activation functoin, please refer to https://keras.io/activations/
model = keras.models.Sequential([Dense(20,activation = "sigmoid",input_shape = (3,)),
                                 Dense(20,activation = "sigmoid"),Dense(20,activation = "sigmoid"),
                                Dense(1)])

# Model summary function shows what you created in the model
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 20)                  │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 20)                  │             420 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 941 (3.68 KB)

 Trainable params: 941 (3.68 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Complie function allows you to choose your measure of loss and optimzer
# For other optimizer, please refer to https://keras.io/optimizers/
model.compile(loss = "mse",optimizer = "Adam")

In [ ]:
# Checkpoint function is used here to periodically save a copy of the model.
# Currently it is set to save the best performing model
checkpoint_cb = keras.callbacks.ModelCheckpoint("implied_vol_model.keras",save_best_only = True)

# Early stopping allows you to stop your training early if no improvment is shown after cerain period
# Currently it is set at if no improvement occured in 1000 epochs, at the stop the model will also revert back to the best weight
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 1000,restore_best_weights = True)

# Remark: checkpoint could be redundant here as early stopping function can also help restoring to the best weight
# We put both here just to illustrate different ways to keep the best model


In [ ]:
# train your model
# The fit function allows you to train a NN model. Here we have training data, number of epochs,batch size, validation data,
# and callbacks as input
# Callback is an optional parameters that allow you to enable tricks for training such as early stopping and checkpoint

# Remarks: Altough we put 50000 epochs here, the model will stop its training once our early stopping criterion is triggered

history=model.fit(X_scaled_train[:,3:6],y_train,epochs=50000, batch_size = 128, verbose = 0, validation_data=(X_scaled_vals[:,3:6],y_val),
                 callbacks=[checkpoint_cb, early_stopping_cb])

KeyboardInterrupt: 

In [ ]:
# Load the best model you saved and calcuate MSE for testing set

model = keras.models.load_model("implied_vol_model.keras")
mse_test = model.evaluate(X_scaled_test[:,3:6],y_test,verbose=0)

print('Test Loss(MSE):', mse_test)

In [ ]:
# Calculate Gain Ratio

gain = 1 - mse_test/mse

print('Gain Ratio:', gain)

## Review your results and export training history


In [ ]:
# Plot training history

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0.00007,0.00015)
plt.show()

#Export your training history for MSE
output = pd.DataFrame(history.history)
output.to_csv("mse_overtime.csv")

In [ ]:
!pip install yfinance
import yfinance as yf
start='2014-06-30'
end='2019-12-31'

vix=yf.download('^VIX')

In [ ]:
vix.plot()